# Задача

Создать рекомендательную систему для пользователей библиотеки с учетом семантики

# Условия задачи

Участникам необходимо для каждого из 16 753 пользователей сделать
подборку из 20 рекомендаций.

Порядок рекомендаций не учитывается, но
очень важно, чтобы рекомендации основывались на интересе пользователя
и были ему релевантны. Обратите внимание, что тестирующая система
принимает только те решения, в которых содержится не более 20
рекомендаций для одного пользователя. Уникальных документов – 354 355.

Участники получают 3 таблицы: users.csv, items.csv,
train_transactions.csv.

Users.csv содержит информацию о читателях, где каждый читатель имеет свой
уникальный номер читательского билета (chb).

Таблица items.csv, содержит описание документов, которые доступны всем читателям,
каждый документ имеет уникальный системный номер (sys_numb).

Таблица train_transactions.csv устанавливает связь между
users-items, показывает наличие взаимодействия читателя с документом.

# Описание данных

##### users.csv:
chb – полный номер читательского билета
age – возраст читателя
gender – пол читателя
chit_type – тип читателя

##### items.csv:
sys_numb – системный номер документа
title – название документа
author – автор документа
izd – издательство
year_izd – год издания
bbk – ББК документа

##### train_transactions.csv:
chb - полный номер читательского билета
sys_numb – системный номер документа
date_1 – дата выдачи
is_real – был ли выдан заказ
type – тип книговыдачи (книговыдача/скачивание)
source – источник (один из трёх онлайн-просмотрщиков)
is_printed – печатный/электронный документ


In [88]:
import gc
import re
import string

import numpy as np
import pandas as pd

import roman

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

import nltk
from nltk.corpus import stopwords as nltk_stopwords

import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Softmax, Dropout, Conv1D, MaxPooling1D

import lightfm
from lightfm.data import Dataset
from lightfm.evaluation import auc_score


from scipy.sparse import csr_matrix

import transformers

from tqdm.notebook import tqdm

In [89]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Шаг 1. Загрузим и посмотрим на данные

In [90]:
users = pd.read_csv(
	'../data/users.csv',
	sep=';',
	index_col=None,
	dtype={'age': str, 'chb': str, 'chit_type': str, 'gender': str}
)

items = pd.read_csv(
	'../data/items.csv',
	sep=';',
	index_col=None,
	dtype={'author': str, 'bbk': str, 'izd': str, 'sys_numb': str, 'title': str, 'year_izd': str}
)

train_transactions = pd.read_csv(
	'../data/train_transactions_extended.csv',
	sep=';',
	index_col=None,
	dtype={'chb': str, 'date_1': str, 'is_printed': str, 'is_real': str, 'source': str, 'sys_numb': str, 'type': str}
)

users.name = 'users'
items.name = 'items'
train_transactions.name = 'train_transactions'

all_data = [users, items, train_transactions]

Руками заполним пропуски в некоторых данных

In [91]:
# у пользователя с индексом 9681 в данных полях стоит значение "отсутствует", заменим его на самое частотное значение поля
users.loc[9681, 'gender'] = users['gender'].value_counts().index[0]
users.loc[9681, 'age'] = users['age'].value_counts().index[0]
users.loc[9681, 'chit_type'] = users['chit_type'].value_counts().index[0]
users.loc[8978, 'age'] = users['age'].value_counts()[0]

# книга без заполненных данных, ее некому будет рекомендовать
items.loc[45417, 'year_izd'] = '2016'
# заполним пропуски в поле bbk
items['bbk'] = items['bbk'].fillna('отсутствует')

# поля "none" и "отсутствует" по сути одно и то же, так что заменим none на отсутствует
items['title'] = items['title'].apply(lambda author: 'отсутствует' if 'none' in author else author)
items['author'] = items['author'].apply(lambda author: 'отсутствует' if 'none' in author else author)
items['izd'] = items['izd'].apply(lambda author: 'отсутствует' if 'none' in author else author)
items['year_izd'] = items['year_izd'].apply(lambda author: 'отсутствует' if 'none' in author else author)
items['bbk'] = items['bbk'].apply(lambda author: 'отсутствует' if 'none' in author else author)

# удаляем квадратные скобки из годов
items['year_izd'].apply(lambda year: year.replace('[', '').replace(']', '')).value_counts()

# создаем фичу определяющую является ли кника учебником или нет
items['is_textboot'] = items['title'].apply(lambda title: int(max([x in title for x in ['учеб', 'Учеб']])))

# типы "скачивание" и "скачка" одинаковые, приводим к единому виду "скачивание"
train_transactions['type'] = train_transactions['type'].apply(lambda type: 'скачивание' if 'скачка' in type else type)

Немного магии с годом издания книги

In [92]:
def is_roman_number(year):
	if '۱۱۴۵' in year:
		return 'отсутствует'

	pattern = r'\d{4}|[I|V|X|L|C|D|M]+'
	res = re.findall(pattern, year)
	if len(res) > 0:
		if len(res) > 1:
			return res[1]
		else:
			return res[0]
	else:
		pattern = r'[а-я]|[А-Я]|[a-z]|[A-Z]'
		res = re.findall(pattern, year)
		if len(res) > 0:
			return 'отсутствует'
		else:
			return year

def from_roman_to_int(year):
	try:
		return str(roman.fromRoman(year))
	except:
		return year

def validate(year):
	try:
		if int(year) >= 2022 or int(year) <= 1000:
			return 'отсутствует'
		else:
			return year
	except:
		return year


def parse(year):
	try:
		return year.split(' ')[0]
	except:
		return year


items['year_izd'] = (items['year_izd']
					.apply(is_roman_number)
					.apply(from_roman_to_int)
					.apply(
						lambda year: year
						.replace('?', '')
						.replace('.', '', 2)
						.replace('-', '0', 2)
						.replace('-', '')
						.replace('[', '')
						.replace(']', '')
					)
					.apply(parse)
					.apply(validate)
					.apply(
						lambda year: year
						.replace('.', '0')
						.replace('–', '0')
						)
					)

Приводим в порядок информацию об издательстве

In [93]:
def check_izdat(izdat):
	if '[б. и.]' in izdat:
		return 'отсутствует'
	elif 'Б. и.' in izdat:
		return 'отсутствует'
	return izdat.replace('[', '').replace(']', '')

items['izd'] = items['izd'].apply(check_izdat)

Приводим в порядок информацию об авторах (увеличивает число дублей среди авторов на 1132 (было 181734, стало 182866)

In [94]:
def get_only_author(author):
	pattern = r'\D+'
	return ' '.join(re.findall(pattern, author)).replace(',', '').replace('-', '')

items['author'] = items['author'].apply(get_only_author)

Создадим таблицу с данными, какие клиенты какие книги уже брали, чтобы их не рекомендовать

In [95]:
books_readed_by_clients = train_transactions.groupby('chb')['sys_numb'].apply(set)

## Подготовим признаки в таблицах

In [96]:
users_data_encoder = OrdinalEncoder()
users_encode_columns = ['gender', 'chit_type']
# users[users_encode_columns] = users_data_encoder.fit_transform(users[users_encode_columns])
users['age'] = users['age'].astype('int')
users.head()

,chb,age,gender,chit_type
0,300001020830,21,female,нет данных
1,300001113642,36,female,нет данных
2,300001148466,46,female,нет данных
3,300001117011,22,female,нет данных
4,200001038094,24,female,echb


In [97]:
kmean_pipe = make_pipeline(
	StandardScaler(),
	KMeans(n_clusters=6)
)
users['user_kmean_class'] = kmean_pipe.fit_predict(users[['age', 'gender', 'chit_type']])
users.sample(7)

ValueError: could not convert string to float: 'female'

In [ ]:
items_data_encoder = OrdinalEncoder()
items_encode_columns = ['author', 'izd', 'bbk']
# items[items_encode_columns] = items_data_encoder.fit_transform(items[items_encode_columns])
items.sample(7)

In [98]:
kmean_pipe = make_pipeline(
	StandardScaler(),
	KMeans(n_clusters=5)
)

items['item_kmean_class'] = kmean_pipe.fit_predict(items[['author', 'izd', 'bbk', 'is_textboot']])
items['year_izd'] = items['year_izd'].apply(lambda year: 0 if 'отсутствует' in year else year)
items.sample(7)

ValueError: could not convert string to float: 'Машовец Асия Океановна'

In [ ]:
# items = items.drop('title', axis=1)
# items.head()

In [ ]:
train_data_encoder = OrdinalEncoder()
train_data_encode_columns = ['is_real', 'type', 'source', 'is_printed']
# train_transactions[train_data_encode_columns] = train_data_encoder.fit_transform(train_transactions[train_data_encode_columns])
# train_transactions = train_transactions.drop('date_1', axis=1)
train_transactions.sample(7)

# EDA

In [99]:
full_data = pd.merge(pd.merge(train_transactions, users, on='chb', how='left'), items, on='sys_numb', how='left')
full_data['year_izd'] = full_data['year_izd'].replace({'': 0})
full_data.head()

,chb,sys_numb,date_1,is_real,type,source,is_printed,age,gender,chit_type,title,author,izd,year_izd,bbk,is_textboot
0,100000641403,RSL01004206702,2021-02-21,yes,скачивание,dlib.rsl.ru,False,55,male,нет данных,Придание огнестойкости деревянным постройкам,Максимов Владимир Лаврович,Гл. упр. землеустройства и земледелия,1915,отсутствует,0
1,100000641403,RSL01000769304,2021-03-23,yes,скачивание,dlib.rsl.ru,False,55,male,нет данных,"Медицинская, научная и общественная деятельнос...",Шалаев Николай Федорович,отсутствует,1993,отсутствует,0
2,100000641403,RSL01004211574,2021-02-21,yes,скачивание,dlib.rsl.ru,False,55,male,нет данных,Кирпичная изба,Максимов Владимир Лаврович,М-во зем.,1916,отсутствует,0
3,100000644359,RSL01009800093,2021-03-16,yes,книговыдача,единый просмоторщик,False,74,male,нет данных,Синтез и свойства новых гетероциклических соед...,Акылбеков Нургали Икрамович,отсутствует,2018,отсутствует,0
4,100000644359,RSL01003557352,2021-03-10,yes,книговыдача,единый просмоторщик,False,74,male,нет данных,Медали в честь Александра Сергеевича Пушкина,Ильин Алексей Алексеевич,тип. А. Бенке,1901,отсутствует,0


In [100]:
def to_lower(data):
	return data.lower()

def remove_round_brackets(data):
  return re.sub('\(.*?\)', '', data)

def remove_punc(data):
  trans = str.maketrans('', '', string.punctuation)
  return data.translate(trans)

def white_space(data):
  return ' '.join(data.split())

def complete_noise(data):
	new_data = to_lower(data)
	new_data = remove_round_brackets(new_data)
	new_data = remove_punc(new_data)
	new_data = white_space(new_data)

	return new_data

items['title'] = items['title'].apply(complete_noise)
items.head()

,sys_numb,title,author,izd,year_izd,bbk,is_textboot
0,RSL01008600016,судебное следствие в уголовном процессе россии...,Машовец Асия Океановна,Юрлитинформ,2016,"Х629.374,0",0
1,RSL01004304880,уральское казачество и его роль в системе росс...,Дубовиков Александр Маратович,отсутствует,2006,отсутствует,0
2,RSL07000461043,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует,0
3,RSL07000433335,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует,0
4,RSL01002419013,я пишу как эхо другого очерки интеллектуал био...,Щедрина Татьяна Геннадьевна,Прогресс-Традиция,2004,"Ю3(2)6-69Шпет Г.Г.-3,0",0


In [116]:
stopwords = set(nltk_stopwords.words('russian'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 3))
tf_idf = count_tf_idf.fit_transform(items['title'])

In [118]:
pd.DataFrame(
	data=tf_idf
).head()

ValueError: Shape of passed values is (354355, 1), indices imply (354355, 3026574)

In [120]:
tf_idf.shape

(354355, 3026574)

In [80]:
items[items['sys_numb'].isin(['RSL01008547404',
						 'RSL01004178414',
						 'RSL01008547404',
						 'RSL01009999463',
						 'RSL01008547404',
						 'RSL01004178414',
						 'RSL01008547404',
						 'RSL01004178414',
						 'RSL01009999463',
						 'RSL01009999463',
						 'RSL01009999463',
						 'RSL01004178414',
						 'RSL01005265875',
						 'RSL01004178414',
						 'RSL01005261958',
						 'RSL01008547404',
						 'RSL01008547404',
						 'RSL01009999463',
						 'RSL01005355619',
						 'RSL01005265873'])]

,sys_numb,title,author,izd,year_izd,bbk,is_textboot
29810,RSL01005265873,Уничтожать на месте трусов и паникеров как пре...,отсутствует,Воениздат,1942,отсутствует,0
48031,RSL01005261958,Памятка по использованию немецких боевых и всп...,отсутствует,Воениздат,1942,отсутствует,0
228916,RSL01005355619,Курс стрельб малокалиберной зенитной артиллери...,отсутствует,Воен. изд-во,1944,отсутствует,0
251458,RSL01005265875,Уничтожение вражеских опорных пунктов : Сб. ст...,отсутствует,"Воениздат, Краснояр. отд-ние",1942,отсутствует,0
262189,RSL01004178414,Особая миссия : повести,Серегин Михаил Георгиевич,Эксмо,2008,Ш5(2=Р)75-644.6,0
287519,RSL01009999463,Волки на воле и взаперти : основано на реальны...,Тамоников Александр Александрович,Эксмо,2019,Ш5(2=Р)64-644.6,0
344874,RSL01008547404,Ненормальная война : 16+,Тамоников Александр Александрович,Эксмо,2016,Ш5(2=Р)64-644.6,0


In [150]:
items[(items['author'] == 'Зверев Сергей Иванович') & (items['izd'] == 'Эксмо')]

,sys_numb,title,author,izd,year_izd,bbk,is_textboot
164980,RSL01006722517,офицерский клинок 16,Зверев Сергей Иванович,Эксмо,2013,Ш5(2=Р)64-644.6,0
182159,RSL01007852930,в волчьей шкуре,Зверев Сергей Иванович,Эксмо,2014,Ш5(2=Р)64-644.6,0
201236,RSL01008894967,лейтенант с одной жизнью 16,Зверев Сергей Иванович,Эксмо,2017,Ш5(2=Р)64-644.6,0
204061,RSL01006683849,порт семи смертей,Зверев Сергей Иванович,Эксмо,2013,Ш5(2=Р)64-644.6,0
205656,RSL01008556936,мы родились в тельняшках роман 16,Зверев Сергей Иванович,Эксмо,2016,Ш5(2=Р)64-644.6,0
213734,RSL01006604393,венесуэльский заговор,Зверев Сергей Иванович,Эксмо,2013,Ш5(2=Р)64-644.6,0
222417,RSL01006626366,по натянутым нервам,Зверев Сергей Иванович,Эксмо,2013,Ш5(2=Р)64-644.6,0
244172,RSL01008252956,скорая десантная помощь 16,Зверев Сергей Иванович,Эксмо,2016,Ш5(2=Р)64-644.6,0
246121,RSL01007514239,день курка 16,Зверев Сергей Иванович,Эксмо,2014,Ш5(2=Р)64-644.6,0
248523,RSL01007526209,зловещие небеса 16,Зверев Сергей Иванович,Эксмо,2014,Ш5(2=Р)64-644.6,0


In [148]:
train_transactions[(train_transactions['chb'] == '200000957291') & (train_transactions['sys_numb'] == 'RSL01006724299')].sort_values(by='date_1')

,chb,sys_numb,date_1,is_real,type,source,is_printed
89626,200000957291,RSL01006724299,2021-02-04,yes,книговыдача,2DL.Viewer,False
89630,200000957291,RSL01006724299,2021-03-11,yes,книговыдача,2DL.Viewer,False
89619,200000957291,RSL01006724299,2021-03-16,yes,книговыдача,2DL.Viewer,False
89613,200000957291,RSL01006724299,2021-03-23,yes,книговыдача,2DL.Viewer,False
89614,200000957291,RSL01006724299,2021-03-25,yes,книговыдача,2DL.Viewer,False
89616,200000957291,RSL01006724299,2021-03-31,yes,книговыдача,2DL.Viewer,False
89622,200000957291,RSL01006724299,2021-04-15,yes,книговыдача,2DL.Viewer,False
89617,200000957291,RSL01006724299,2021-04-21,yes,книговыдача,2DL.Viewer,False
89620,200000957291,RSL01006724299,2021-04-29,yes,книговыдача,2DL.Viewer,False
89615,200000957291,RSL01006724299,2021-05-12,yes,книговыдача,2DL.Viewer,False


In [149]:
full_data.groupby(['chb', 'izd', 'author']).count()['is_real'].sort_values(ascending=False)

chb           izd           author                        
200001017435  отсутствует   отсутствует                       806
300001058749  отсутствует   отсутствует                       766
300000863416  отсутствует   отсутствует                       724
300001030290  отсутствует   отсутствует                       676
100001073170  отсутствует   отсутствует                       588
                                                             ... 
200000952623  отсутствует   Шалаев Андрей Викторович            1
200000952625  Академия      Гиленсон Борис Александрович        1
              Аспект пресс  Лазутина Галина Викторовна          1
              Бомбора       Наказава Донна Джексон              1
400001035059  отсутствует   Штибен Владимир Константинович      1
Name: is_real, Length: 195420, dtype: int64

# END EDA


In [14]:
full_data = pd.merge(pd.merge(train_transactions, users, on='chb', how='left'), items, on='sys_numb', how='left')
full_data['year_izd'] = full_data['year_izd'].replace({'': 0})
full_data.head()

,chb,sys_numb,is_real,type,source,is_printed,age,gender,chit_type,user_kmean_class,author,izd,year_izd,bbk,is_textboot,item_kmean_class
0,100000641403,RSL01004206702,1.0,2.0,2.0,0.0,55,1.0,2.0,3,90012.0,6678.0,1915,130777.0,0,0
1,100000641403,RSL01000769304,1.0,2.0,2.0,0.0,55,1.0,2.0,3,162326.0,33897.0,1993,130777.0,0,3
2,100000641403,RSL01004211574,1.0,2.0,2.0,0.0,55,1.0,2.0,3,90012.0,17263.0,1916,130777.0,0,0
3,100000644359,RSL01009800093,1.0,1.0,3.0,0.0,74,1.0,2.0,3,4695.0,33897.0,2018,130777.0,0,1
4,100000644359,RSL01003557352,1.0,1.0,3.0,0.0,74,1.0,2.0,3,59405.0,34398.0,1901,130777.0,0,1


In [15]:
ids = full_data[['chb', 'sys_numb']]
features_tensor = tf.convert_to_tensor(full_data.drop(['chb', 'sys_numb'], axis=1).astype('float32'))


In [16]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(features_tensor)

In [17]:
features_tensor.shape

TensorShape([259566, 14])

In [18]:
def create_model():
	optimizer = tf.optimizers.Adam(lr=0.0001)
	model = Sequential()
	model.add(normalizer)
	# model.add(Conv1D(filters=32, kernel_size=2))
	# model.add(MaxPooling1D())
	# model.add(LSTM(units=128, activation='relu'))
	# model.add(Dropout(0.1))
	# model.add(LSTM(units=64, activation='relu'))
	# model.add(Dropout(0.1))
	model.add(Dense(units=128, activation='relu'))
	model.add(Dropout(0.1))
	model.add(Dense(units=1, activation='sigmoid'))

	model.compile(optimizer=optimizer, loss=tf.losses.MeanSquaredError())

	return model

In [19]:
# full_data['year_izd'] = full_data['year_izd'].apply(lambda year: 0 if '' in year else year)

In [20]:
new_full_data = full_data.drop(['chb', 'sys_numb'], axis=1)
new_full_data['year_izd'] = new_full_data['year_izd'].astype(np.int32)

In [21]:
user_iteractions = pd.DataFrame(
	np.empty(shape=(len(set(users['chb'].values)), len(set(items['sys_numb'].values))), dtype=np.int8),
	index=list(set(users['chb'].values)),
	columns=list(set(items['sys_numb'].values)),
)

In [22]:
user_iteractions.head()

,RSL01010345388,RSL01005502246,RSL07000427853,RSL01000600561,RSL01003059261,RSL02000020386,RSL01003039941,RSL01008957427,RSL01004421075,RSL01008949627,...,RSL01005785725,RSL01003654602,RSL01002570593,RSL01003370055,RSL01008530444,RSL01004584011,RSL01003110172,RSL01005032616,RSL01006501986,RSL01003087622
300000831483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
300000961823,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
300001030141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
200001076347,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
300001028859,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
for user in tqdm(list(set(users['chb'].values))):
	user_readed_books = list(set(train_transactions[train_transactions['chb'] == user]['sys_numb'].values))
	for read_book_user in user_readed_books:
		user_iteractions.loc[user, read_book_user] = 1

  0%|          | 0/16753 [00:00<?, ?it/s]

In [24]:
# users_features = users.drop('chb', axis=1)
# items_features = items.drop(['sys_numb', 'title'], axis=1)
# items_features['year_izd'] = items_features['year_izd'].apply(lambda year: 0 if 'отсутствует' in year else year)

In [25]:
user_ids = users['chb']
item_ids = items['sys_numb']

In [26]:
# user_iteractions = user_iteractions.replace({0: -1})

In [27]:
def create_features(dataframe, features_name, id_col_name):
    """
    Generate features that will be ready for feeding into lightfm

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe which contains features
    features_name : List
        List of feature columns name avaiable in dataframe
    id_col_name: String
        Column name which contains id of the question or
        answer that the features will map to.
        There are two possible values for this variable.
        1. questions_id_num
        2. professionals_id_num

    Returns
    -------
    Pandas Series
        A pandas series containing process features
        that are ready for feed into lightfm.
        The format of each value
        will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
        Ex. -> (1, ['military', 'army', '5'])
    """

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features



def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A.
    features_name : List
        List of feature columns name avaiable in dataframe.

    Returns
    -------
    List of all features for mapping
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features

In [28]:
users_features = create_features(users, ['age', 'gender', 'chit_type', 'user_kmean_class'], 'chb')
item_features = create_features(items, ['author', 'izd', 'year_izd', 'bbk', 'is_textboot', 'item_kmean_class'], 'sys_numb')

In [29]:
users_features_list = generate_feature_list(users, ['age', 'gender', 'chit_type', 'user_kmean_class'])
items_features_list = generate_feature_list(items, ['author', 'izd', 'year_izd', 'bbk', 'is_textboot', 'item_kmean_class'])

In [30]:
dataset = Dataset()
dataset.fit(
	set(users['chb'].values),
	set(items['sys_numb'].values),
	item_features=items_features_list,
	user_features=users_features_list
)

In [31]:
res = []
for user_id_iterator in tqdm(user_iteractions.index.values):
	res.append(1 / user_iteractions.loc[str(user_id_iterator)].sum())

total_weights = pd.Series(res)

  0%|          | 0/16753 [00:00<?, ?it/s]

In [32]:
iteractions_matrix = list(
	zip(
		users['chb'].values,
		items['sys_numb'].values,
		total_weights
	)
)

interactions, weights = dataset.build_interactions(iteractions_matrix)

last_item_features = dataset.build_item_features(item_features)

last_user_features = dataset.build_user_features(users_features)

In [33]:
model = lightfm.LightFM(
	no_components=150,
	learning_rate=0.05,
	loss='warp',
	random_state=25
)

model.fit(
	interactions,
	item_features=last_item_features,
	user_features=last_user_features,
	sample_weight=weights,
	epochs=5,
	num_threads=16,
	verbose=True
)

Epoch: 100%|██████████| 5/5 [00:06<00:00,  1.33s/it]


In [34]:
def calculate_auc_score(lightfm_model, interactions_matrix,
                        question_features, professional_features):
    """
    Measure the ROC AUC metric for a model.
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model
        A fitted lightfm model
    interactions_matrix :
        A lightfm interactions matrix
    question_features, professional_features:
        Lightfm features

    Returns
    -------
    String containing AUC score
    """
    score = auc_score(
        lightfm_model, interactions_matrix,
        item_features=question_features,
        user_features=professional_features,
        num_threads=16
	).mean()
    return score

In [35]:
# не смог дождаться окончания выполнения
# calculate_auc_score(model, interactions, last_item_features, last_user_features)

In [36]:
users.head()

,chb,age,gender,chit_type,user_kmean_class
0,300001020830,21,0.0,2.0,0
1,300001113642,36,0.0,2.0,0
2,300001148466,46,0.0,2.0,5
3,300001117011,22,0.0,2.0,0
4,200001038094,24,0.0,0.0,2


In [56]:
res_list = []

for index in tqdm(list(users.index)):
	uniq_user = users.loc[index, 'chb']

	preds = model.predict(
		index,
		items['sys_numb'].index.to_list(),
		item_features=last_item_features,
		user_features=last_user_features
	)
	preds = items.loc[pd.Series(preds).sort_values(ascending=False)[:20].index]['sys_numb']

	for pred in preds:
		res_list.append([uniq_user, pred])

  0%|          | 0/16753 [00:00<?, ?it/s]

In [57]:
pd.DataFrame(res_list, columns=['chb', 'sys_numb']).to_csv('5th_iter.csv', sep=';', index=False)